In [2]:
import matplotlib.pyplot as plt
import xarray as xr
import pandas as pd
from pyclim_noresm.general_util_funcs import global_avg
import matplotlib

In [5]:
paths = sorted(snakemake.input.paths)
vname = snakemake.wildcards.vName

In [16]:
erfs = {}
erfs['mean'] = {}
erfs['std'] = {}
for path in paths:
    ds = xr.open_dataset(path)
    source = ds.parent_source_id
    ts = ds.mean(dim='year')
    ts = global_avg(ts[vname])
    std = global_avg(ds[vname])
    std = std.std(dim='year')

    erfs['mean'][source] = ts.values
    erfs['std'][source] = std.values
    

In [18]:
df = pd.DataFrame(erfs['mean'],index=['mean'])

In [19]:
df.loc['mean','Multi-model'] = df.loc['mean',:].mean()
df = df.transpose()

In [20]:
vmax=round(abs(df.loc[:,'mean']).max()+0.1,1)

In [21]:
fig, ax = plt.subplots(figsize=(6,6))
ax.axvline(color='darkgrey')
df.plot.barh(ax=ax, legend=False)
ax.set_xlim(-vmax,vmax)
ax.set_xlabel('$W/m^2$')
plt.savefig(snakemake.output.outpath, bbox_inches='tight', facecolor='white')